In [20]:
import pandas as pd
import re

In [2]:
events = r"faq.jsonl"

with open(events, "r", encoding="utf-8") as f:
    events_df = pd.read_json(f, lines=True)

In [9]:
sampling_df = events_df[events_df.type == "sampling"].reset_index(drop=True)
sampling_df = sampling_df.join(pd.json_normalize(sampling_df.data)).explode("sampled")


In [33]:
error_sampling = sampling_df[sampling_df.sampled.str.endswith("N")]
error_sampling

,spec,run_id,event_id,sample_id,type,data,created_by,created_at,final_report,prompt,sampled
10,NaN,2306081515275J4ZJVPB,12.0,faq-closedqa-relevance.dev.9,sampling,"{'prompt': [{'role': 'user', 'content': 'You a...",,2023-06-08 15:16:08.225338+00:00,NaN,"[{'role': 'user', 'content': 'You are assessin...",The criterion is asking whether the submission...
31,NaN,2306081515275J4ZJVPB,44.0,faq-closedqa-relevance.dev.9,sampling,"{'prompt': [{'role': 'user', 'content': 'You a...",,2023-06-08 15:16:54.667049+00:00,NaN,"[{'role': 'user', 'content': 'You are assessin...",The criterion is asking whether the submission...
51,NaN,2306081515275J4ZJVPB,75.0,faq-closedqa-relevance.dev.9,sampling,"{'prompt': [{'role': 'user', 'content': 'You a...",,2023-06-08 15:17:36.924326+00:00,NaN,"[{'role': 'user', 'content': 'You are assessin...",The criterion is asking whether the submission...
72,NaN,2306081515275J4ZJVPB,107.0,faq-closedqa-relevance.dev.9,sampling,"{'prompt': [{'role': 'user', 'content': 'You a...",,2023-06-08 15:18:17.155340+00:00,NaN,"[{'role': 'user', 'content': 'You are assessin...",The criterion is asking whether the submission...
104,NaN,2306081515275J4ZJVPB,155.0,faq-closedqa-relevance.dev.9,sampling,"{'prompt': [{'role': 'user', 'content': 'You a...",,2023-06-08 15:19:57.141022+00:00,NaN,"[{'role': 'user', 'content': 'You are assessin...",The criterion is asking whether the submission...


In [25]:
error_sampling.sampled.str.find(r"User_Question:(.*)Answer:", re.MULTILINE)

10    -1
31    -1
51    -1
72    -1
104   -1
Name: sampled, dtype: int64

In [48]:
user_question = re.compile(r"User_Question: (.*)", re.MULTILINE)
answer = re.compile(r"\[Submission\]: (.*)", re.MULTILINE)
for i, r in error_sampling.iterrows():
    # print(f"sample_id: {r.sample_id}")
    # print(f"event_id: {r.event_id}")
    # print(f"Prompt: {r.prompt[0]}")
    print(f"User Question: {re.search(user_question, r.prompt[0]['content']).group(1)}")
    print(f"Answer: {re.search(answer, r.prompt[0]['content']).group(1)}")
    print(f"Reason: {r.sampled}")
    # print(r.prompt[0]['content'])
    # print(f"Sampled: {r.sampled}")
    print("-" * 25)

User Question: 別の Windows 365 プランへのアップグレードやダウングレードは可能ですか?
Answer: Windows 365 のお客様は、"サイズ変更" 機能を使用して RAM、CPU、ストレージのサイズをユーザーのニーズに合わせてアップグレードできます。たとえば、CPU 多用型のアプリを実行するユーザーがさらにパワフルなクラウド PC を必要としている場合や、ファイル保存用に追加のディスク領域を必要としている場合です。ダウングレードのオプションは、現時点では用意されていません。
Reason: The criterion is asking whether the submission refers to a real quote from the text. The text provides information about the ability to upgrade the size of RAM, CPU, and storage using the "size change" feature in Windows 365, but it does not mention anything about upgrading or downgrading to a different plan. The submission mentions the "size change" feature and talks about upgrading, but it does not mention anything about downgrading or changing to a different plan. Therefore, the submission does not meet the criterion.

Answer: N

N
-------------------------
User Question: 別の Windows 365 プランへのアップグレードやダウングレードは可能ですか?
Answer: Windows 365 のお客様は、"サイズ変更" 機能を使用して RAM、CPU、ストレージのサイズをユーザーのニーズに合わせてアップグレードできます。たとえば、CPU 多用型のアプリを実行するユーザ